## Downloading menu card from Facebook

In [17]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import requests
import json
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
from deep_translator import GoogleTranslator
from sqlalchemy import create_engine

os.environ['KMP_DUPLICATE_LIB_OK']='True'
import easyocr

database_credential = json.load(open('credentials.json'))
connection = create_engine(database_credential['db'])

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.facebook.com/chillox.burgers/photos/p.4362603440501202/4362603440501202')
print ("Opened facebook")
time.sleep(1)

image_path = "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div/div[1]/div/div[1]/div/div[2]/div/div/div/img"

# get the image source
img = driver.find_element_by_xpath(image_path)
src = img.get_attribute('src')

# download the image
img_data = requests.get(src).content
with open('./data/3_chillox_menu.jpg', 'wb') as handler:
    handler.write(img_data)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\my pc\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


Opened facebook


## Extracting name and price using OCR (Optical Character Recognition)

In [4]:
# This process is a bit slow. Please wait patiently

IMAGE_PATH = './data/3_chillox_menu.jpg'
reader = easyocr.Reader(['en'])
result = reader.readtext(IMAGE_PATH,paragraph="False")
test = pd.DataFrame({'name': result})

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [5]:
name_price_list = []

for i in range(len(test)):
    name_price = test['name'].iloc[i][-1]
    name_price_list.append(name_price)
    
testing = pd.DataFrame({'name': name_price_list})

#### OCR caused a few messy data, cleaning them

In [6]:
price_list = []
product_list = []

for i in range(len(testing)):
    if "TK" in testing['name'].iloc[i]:
        price = (testing['name'].iloc[i])
        
        condition = (price.split(" ")[-1])
        
        if condition == "TK":
            
            name = testing['name'].iloc[i-1]
            price = testing['name'].iloc[i]
            
            try:
                price = int(price.split(" ")[0])
                
                product_list.append(name)
                price_list.append(price)
                
            except:
                pass

chillox = pd.DataFrame({'name': product_list, 'price': price_list})
chillox['restaurant'] = None
chillox['restaurant'] = "chillox"
chillox.to_sql(con=connection, name = 'chillox', if_exists='replace', index=False)

chillox.tail(3)

,name,price,restaurant
9,FRENCH FRIES (REGULARILARGE),70,chillox
10,BINGE BEEFICHICKEN 2x Giant Beeli Chicken Patl...,680,chillox
11,CHICKEN FINGERS (5 PCS/1O PCS),130,chillox


## Extracting location data

In [15]:
path = 'https://www.google.com/search?q=chillox+location&rlz=1C1ONGR_enBD1012BD1012&biw=1036&bih=666&sz=0&tbm=lcl&ei=c3w0Y4XpA5WOseMPx6eniAw&ved=0ahUKEwjF7sr_-bf6AhUVR2wGHcfTCcEQ4dUDCAk&uact=5&oq=chillox+location&gs_lcp=Cg1nd3Mtd2l6LWxvY2FsEAMyBAgAEBMyCAgAEB4QFhATMggIABAeEBYQEzIICAAQHhAWEBMyCAgAEB4QFhATMggIABAeEBYQEzIICAAQHhAWEBMyCAgAEB4QFhATMggIABAeEBYQEzIICAAQHhAWEBNQAFgAYMMOaABwAHgAgAGtAYgBrQGSAQMwLjGYAQDAAQE&sclient=gws-wiz-local#rlfi=hd:;si:;mv:[[23.817414,90.42820069999999],[23.713757800000003,90.35701499999999]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:4'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(path)
all_location = driver.find_element("id", 'rl_ist0')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\my pc\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


#### Taking just location(Chillox Mirpur > Mirpur)

#### Converting bangla location to eng using GoogleTranslator

In [16]:
location_list_uncleaned = all_location.text.split("\n")

cleaned_location = []

for loc in location_list_uncleaned:
    if "চিলক্স" in loc:
        loc = loc.replace("চিলক্স", "").strip()
        loc = GoogleTranslator(source='auto', target='en').translate(loc) 
        cleaned_location.append(loc)
    elif 'Chillox' in loc:
        loc = loc.replace('Chillox', "").strip()
        cleaned_location.append(loc)   

location_data = pd.DataFrame({'location': cleaned_location})
location_data['restaurant'] = None
location_data['restaurant'] = "chillox"

location_data

,location,restaurant
0,Dhanmondi,chillox
1,Khilgaon,chillox
2,Wari,chillox
3,Mohammadpur,chillox
4,great empty,chillox
5,Aftabnagar,chillox
6,Banani,chillox
7,Basundhara,chillox


In [14]:
location_data.to_sql(con=connection, name = 'location_data', if_exists='append', index=False)